In [6]:
# Improved 2D Tree Species Classification Pipeline
# Multiple strategies to boost accuracy from 70% closer to 84%

import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import time

# Machine Learning imports
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE  # pip install imbalanced-learn

# Computer Vision imports
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage import exposure, filters
from skimage.measure import shannon_entropy
from skimage.segmentation import slic
from skimage.color import rgb2gray

# Progress tracking
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

# =============================================================================
# STRATEGY 1: Enhanced LBP Feature Extraction (Multiple Scales & Parameters)
# =============================================================================

def extract_enhanced_lbp_features(image_path, lbp_configs=None):
    """
    Extract LBP features with multiple configurations for richer representation.
    """
    if lbp_configs is None:
        # Multiple LBP configurations for multi-scale analysis
        lbp_configs = [
            {'P': 8, 'R': 1, 'method': 'uniform'},    # Fine details
            {'P': 16, 'R': 2, 'method': 'uniform'},   # Medium details  
            {'P': 24, 'R': 3, 'method': 'uniform'},   # Coarse details
        ]
    
    try:
        # Load image in grayscale
        image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            return None
            
        # Apply slight Gaussian blur to reduce noise
        image = cv2.GaussianBlur(image, (3, 3), 0)
        
        all_features = []
        
        for config in lbp_configs:
            P, R, method = config['P'], config['R'], config['method']
            
            # Compute LBP
            lbp = local_binary_pattern(image, P, R, method=method)
            
            # Calculate histogram
            bins = P + 2 if method == 'uniform' else 2**P
            hist, _ = np.histogram(lbp.ravel(), bins=bins, range=(0, bins))
            
            # Normalize histogram
            hist = hist.astype(float)
            if hist.sum() > 0:
                hist /= hist.sum()
                
            all_features.append(hist)
        
        # Concatenate all LBP features
        combined_features = np.concatenate(all_features)
        return combined_features
        
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

# =============================================================================
# STRATEGY 2: Alternative Feature Extraction Methods
# =============================================================================

def extract_glcm_features(image_path, distances=[1, 2], angles=[0, 45, 90, 135]):
    """
    Extract Gray-Level Co-occurrence Matrix (GLCM) features.
    """
    try:
        image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            return None
            
        # Reduce gray levels for GLCM computation (speeds up and reduces noise)
        image = (image // 32).astype(np.uint8)  # 8 gray levels
        
        features = []
        
        for distance in distances:
            for angle in angles:
                # Compute GLCM
                glcm = graycomatrix(image, [distance], [np.radians(angle)], 
                                 levels=8, symmetric=True, normed=True)
                
                # Extract texture properties
                contrast = graycoprops(glcm, 'contrast')[0, 0]
                dissimilarity = graycoprops(glcm, 'dissimilarity')[0, 0]
                homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
                energy = graycoprops(glcm, 'energy')[0, 0]
                correlation = graycoprops(glcm, 'correlation')[0, 0]
                
                features.extend([contrast, dissimilarity, homogeneity, energy, correlation])
        
        return np.array(features)
        
    except Exception as e:
        print(f"Error processing GLCM for {image_path}: {e}")
        return None

def extract_statistical_features(image_path):
    """
    Extract basic statistical features from the image.
    """
    try:
        image = cv2.imread(str(image_path), cv2.IMREAD_GRAYSCALE)
        if image is None:
            return None
            
        features = []
        
        # Basic statistics
        features.append(np.mean(image))
        features.append(np.std(image))
        features.append(np.var(image))
        features.append(shannon_entropy(image))
        
        # Gradient magnitude statistics
        grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
        grad_mag = np.sqrt(grad_x**2 + grad_y**2)
        
        features.append(np.mean(grad_mag))
        features.append(np.std(grad_mag))
        
        return np.array(features)
        
    except Exception as e:
        print(f"Error processing statistical features for {image_path}: {e}")
        return None

def extract_combined_features(image_path, feature_types=['enhanced_lbp', 'glcm', 'statistical']):
    """
    Extract and combine multiple types of features.
    """
    all_features = []
    
    if 'enhanced_lbp' in feature_types:
        lbp_feat = extract_enhanced_lbp_features(image_path)
        if lbp_feat is not None:
            all_features.append(lbp_feat)
    
    if 'glcm' in feature_types:
        glcm_feat = extract_glcm_features(image_path)
        if glcm_feat is not None:
            all_features.append(glcm_feat)
    
    if 'statistical' in feature_types:
        stat_feat = extract_statistical_features(image_path)
        if stat_feat is not None:
            all_features.append(stat_feat)
    
    if all_features:
        return np.concatenate(all_features)
    else:
        return None

# =============================================================================
# STRATEGY 3: Improved Multi-view Aggregation
# =============================================================================

def extract_multiview_features_improved(grouped_data, feature_types=['enhanced_lbp'], 
                                       aggregation_methods=['mean']):
    """
    Extract features with multiple aggregation strategies.
    """
    X_features = []
    y_labels = []
    
    print(f"Extracting features with types: {feature_types}")
    print(f"Aggregation methods: {aggregation_methods}")
    
    # Get feature dimension by testing on first image
    first_species = list(grouped_data.keys())[0]
    first_tree = list(grouped_data[first_species].keys())[0]
    first_image = grouped_data[first_species][first_tree][0]
    test_features = extract_combined_features(first_image, feature_types)
    
    if test_features is None:
        raise ValueError("Could not extract features from test image")
    
    feature_dim = len(test_features)
    print(f"Feature dimension per image: {feature_dim}")
    
    # Calculate total feature dimension based on aggregation methods
    total_dim = feature_dim * len(aggregation_methods)
    print(f"Total feature dimension per tree: {total_dim}\n")
    
    for species_name, trees in tqdm(grouped_data.items(), desc="Processing species"):
        for tree_name, image_paths in tqdm(trees.items(), desc=f"{species_name} trees", leave=False):
            
            # Extract features from all views of this tree
            tree_features = []
            
            for img_path in image_paths:
                features = extract_combined_features(img_path, feature_types)
                if features is not None:
                    tree_features.append(features)
            
            # Aggregate features using multiple methods
            if tree_features:
                tree_features_array = np.stack(tree_features)
                aggregated_features = []
                
                for agg_method in aggregation_methods:
                    if agg_method == 'mean':
                        agg_feat = np.mean(tree_features_array, axis=0)
                    elif agg_method == 'std':
                        agg_feat = np.std(tree_features_array, axis=0)
                    elif agg_method == 'max':
                        agg_feat = np.max(tree_features_array, axis=0)
                    elif agg_method == 'min':
                        agg_feat = np.min(tree_features_array, axis=0)
                    else:
                        agg_feat = np.mean(tree_features_array, axis=0)  # default
                    
                    aggregated_features.append(agg_feat)
                
                final_features = np.concatenate(aggregated_features)
                X_features.append(final_features)
                y_labels.append(species_name)
    
    X_features = np.array(X_features)
    y_labels = np.array(y_labels)
    
    # Create label encoder
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y_labels)
    
    print(f"Feature extraction complete!")
    print(f"Feature matrix shape: {X_features.shape}")
    print(f"Species: {list(label_encoder.classes_)}")
    print(f"Label distribution: {dict(zip(*np.unique(y_labels, return_counts=True)))}\n")
    
    return X_features, y_encoded, label_encoder

# =============================================================================
# STRATEGY 4: Comprehensive Grid Search with SMOTE and Class Balancing
# =============================================================================

def comprehensive_model_search(X_train, y_train, X_test, y_test, label_encoder):
    """
    Comprehensive model search similar to your 3D approach.
    """
    print("="*60)
    print("COMPREHENSIVE MODEL SEARCH")
    print("="*60)
    
    # Feature configurations to test
    feature_configs = [
        {
            'name': 'Enhanced_LBP',
            'types': ['enhanced_lbp'],
            'aggregation': ['mean']
        },
        {
            'name': 'LBP+GLCM',
            'types': ['enhanced_lbp', 'glcm'],
            'aggregation': ['mean']
        },
        {
            'name': 'All_Features',
            'types': ['enhanced_lbp', 'glcm', 'statistical'],
            'aggregation': ['mean']
        },
        {
            'name': 'Multi_Aggregation',
            'types': ['enhanced_lbp'],
            'aggregation': ['mean', 'std']
        }
    ]
    
    best_summary = None
    summaries = []
    
    for config in feature_configs:
        print(f"\n{'='*60}")
        print(f"Testing: {config['name']}")
        print(f"Features: {config['types']}")
        print(f"Aggregation: {config['aggregation']}")
        print("="*60)
        
        try:
            # Re-extract features with this configuration
            print("Re-extracting training features...")
            X_train_config, y_train_config, _ = extract_multiview_features_improved(
                train_grouped, config['types'], config['aggregation'])
            
            print("Re-extracting test features...")
            X_test_config, y_test_config, _ = extract_multiview_features_improved(
                test_grouped, config['types'], config['aggregation'])
            
            # SMOTE to balance classes (following your 3D approach)
            print("Applying SMOTE...")
            smote = SMOTE(random_state=42)
            X_train_bal, y_train_bal = smote.fit_resample(X_train_config, y_train_config)
            print(f"After SMOTE: {X_train_config.shape} -> {X_train_bal.shape}")
            
            # Scale features
            print("Scaling features...")
            scaler = StandardScaler()
            X_train_scaled = scaler.fit_transform(X_train_bal)
            X_test_scaled = scaler.transform(X_test_config)
            
            # Comprehensive parameter grid (following your 3D approach)
            param_grid = {
                'C': [0.1, 1, 3, 10, 30, 100],
                'gamma': ['scale', 'auto', 0.03, 0.1, 0.3, 1],
                'class_weight': [None, 'balanced'],
                'kernel': ['rbf']
            }
            
            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            grid = GridSearchCV(
                SVC(random_state=42), 
                param_grid, 
                cv=cv, 
                n_jobs=-1, 
                verbose=0, 
                scoring='balanced_accuracy'  # Use balanced_accuracy like your 3D approach
            )
            
            print("Running grid search...")
            start = time.time()
            grid.fit(X_train_scaled, y_train_bal)
            dur = time.time() - start
            
            print(f"Grid completed in {dur:.1f}s")
            print(f"Best balanced accuracy (CV): {grid.best_score_:.4f}")
            print(f"Best parameters: {grid.best_params_}")
            
            # Evaluate on test set
            best_svm = grid.best_estimator_
            y_pred = best_svm.predict(X_test_scaled)
            
            test_acc = accuracy_score(y_test_config, y_pred)
            test_bal_acc = balanced_accuracy_score(y_test_config, y_pred)
            
            print(f"Test accuracy: {test_acc:.4f}")
            print(f"Test balanced accuracy: {test_bal_acc:.4f}")
            
            # Store results
            summary = {
                'config_name': config['name'],
                'feature_types': config['types'],
                'aggregation': config['aggregation'],
                'feature_dim': X_train_config.shape[1],
                'best_params': grid.best_params_,
                'cv_balanced_accuracy': grid.best_score_,
                'test_accuracy': test_acc,
                'test_balanced_accuracy': test_bal_acc,
                'y_pred': y_pred,
                'y_true': y_test_config
            }
            summaries.append(summary)
            
            # Track best configuration
            if best_summary is None or summary['cv_balanced_accuracy'] > best_summary['cv_balanced_accuracy']:
                best_summary = summary
                
        except Exception as e:
            print(f"Error with configuration {config['name']}: {e}")
            continue
    
    return best_summary, summaries

# =============================================================================
# LOAD DATA (using previous grouping functions)
# =============================================================================

# Re-use your existing grouped data
print("Using previously grouped data...")
print(f"Training: {len(sum([list(trees.keys()) for trees in train_grouped.values()], []))} trees")
print(f"Test: {len(sum([list(trees.keys()) for trees in test_grouped.values()], []))} trees")

# =============================================================================
# RUN COMPREHENSIVE SEARCH
# =============================================================================

best_config, all_summaries = comprehensive_model_search(
    None, None, None, None, None  # We'll extract features inside the function
)

# =============================================================================
# DISPLAY RESULTS
# =============================================================================

print("\n" + "="*60)
print("FINAL RESULTS COMPARISON")
print("="*60)

results_df = pd.DataFrame([{
    'Configuration': s['config_name'],
    'Feature_Dim': s['feature_dim'],
    'CV_Balanced_Acc': f"{s['cv_balanced_accuracy']:.4f}",
    'Test_Accuracy': f"{s['test_accuracy']:.4f}",
    'Test_Balanced_Acc': f"{s['test_balanced_accuracy']:.4f}",
    'Best_C': s['best_params']['C'],
    'Best_Gamma': s['best_params']['gamma'],
    'Class_Weight': s['best_params']['class_weight']
} for s in all_summaries])

print(results_df.to_string(index=False))

print(f"\n{'='*60}")
print("BEST CONFIGURATION DETAILS")
print("="*60)
if best_config:
    print(f"Configuration: {best_config['config_name']}")
    print(f"Feature types: {best_config['feature_types']}")
    print(f"Aggregation: {best_config['aggregation']}")
    print(f"Feature dimension: {best_config['feature_dim']}")
    print(f"Best parameters: {best_config['best_params']}")
    print(f"CV Balanced Accuracy: {best_config['cv_balanced_accuracy']:.4f}")
    print(f"Test Accuracy: {best_config['test_accuracy']:.4f}")
    print(f"Test Balanced Accuracy: {best_config['test_balanced_accuracy']:.4f}")
    
    print(f"\nCLASSIFICATION REPORT (Best Configuration):")
    print("-" * 50)
    target_names = [name.replace('np.str_(\'', '').replace('\')', '') for name in 
                   ['Ash', 'Beech', 'Douglas Fir', 'Oak', 'Pine', 'Red Oak', 'Spruce']]
    print(classification_report(best_config['y_true'], best_config['y_pred'], 
                              target_names=target_names))
    
    print(f"\nCONFUSION MATRIX (Best Configuration):")
    print("-" * 50)
    cm = confusion_matrix(best_config['y_true'], best_config['y_pred'])
    cm_df = pd.DataFrame(cm, index=target_names, columns=target_names)
    print(cm_df)

print(f"\n{'='*60}")
print("COMPARISON WITH YOUR 3D RESULTS")
print("="*60)
print("Your 3D FPFH + SMOTE + RBF SVM: 84% accuracy")
print(f"Best 2D approach: {best_config['test_accuracy']:.1%} accuracy" if best_config else "No valid results")
print(f"Improvement strategies applied:")
print("✓ SMOTE for class balancing")
print("✓ Balanced accuracy optimization")
print("✓ Class weight parameter")
print("✓ Enhanced feature extraction")
print("✓ Comprehensive grid search")
print("✓ Multi-scale LBP features")
print("✓ Additional texture features (GLCM)")
print("="*60)

All libraries imported successfully!
Using previously grouped data...
Training: 1114 trees
Test: 268 trees
COMPREHENSIVE MODEL SEARCH

Testing: Enhanced_LBP
Features: ['enhanced_lbp']
Aggregation: ['mean']
Re-extracting training features...
Extracting features with types: ['enhanced_lbp']
Aggregation methods: ['mean']
Feature dimension per image: 54
Total feature dimension per tree: 54



Processing species: 100%|██████████| 7/7 [00:51<00:00,  7.40s/it]


Feature extraction complete!
Feature matrix shape: (1114, 54)
Species: [np.str_('Ash'), np.str_('Beech'), np.str_('Douglas Fir'), np.str_('Oak'), np.str_('Pine'), np.str_('Red Oak'), np.str_('Spruce')]
Label distribution: {np.str_('Ash'): np.int64(64), np.str_('Beech'): np.int64(264), np.str_('Douglas Fir'): np.int64(294), np.str_('Oak'): np.int64(36), np.str_('Pine'): np.int64(40), np.str_('Red Oak'): np.int64(162), np.str_('Spruce'): np.int64(254)}

Re-extracting test features...
Extracting features with types: ['enhanced_lbp']
Aggregation methods: ['mean']
Feature dimension per image: 54
Total feature dimension per tree: 54



Processing species: 100%|██████████| 7/7 [00:12<00:00,  1.78s/it]


Feature extraction complete!
Feature matrix shape: (268, 54)
Species: [np.str_('Ash'), np.str_('Beech'), np.str_('Douglas Fir'), np.str_('Oak'), np.str_('Pine'), np.str_('Red Oak'), np.str_('Spruce')]
Label distribution: {np.str_('Ash'): np.int64(14), np.str_('Beech'): np.int64(64), np.str_('Douglas Fir'): np.int64(72), np.str_('Oak'): np.int64(8), np.str_('Pine'): np.int64(10), np.str_('Red Oak'): np.int64(38), np.str_('Spruce'): np.int64(62)}

Applying SMOTE...
After SMOTE: (1114, 54) -> (2058, 54)
Scaling features...
Running grid search...
Grid completed in 5.6s
Best balanced accuracy (CV): 0.9339
Best parameters: {'C': 100, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}
Test accuracy: 0.7388
Test balanced accuracy: 0.7244

Testing: LBP+GLCM
Features: ['enhanced_lbp', 'glcm']
Aggregation: ['mean']
Re-extracting training features...
Extracting features with types: ['enhanced_lbp', 'glcm']
Aggregation methods: ['mean']
Feature dimension per image: 94
Total feature dimension p

...

Feature extraction complete!
Feature matrix shape: (268, 100)
Species: [np.str_('Ash'), np.str_('Beech'), np.str_('Douglas Fir'), np.str_('Oak'), np.str_('Pine'), np.str_('Red Oak'), np.str_('Spruce')]
Label distribution: {np.str_('Ash'): np.int64(14), np.str_('Beech'): np.int64(64), np.str_('Douglas Fir'): np.int64(72), np.str_('Oak'): np.int64(8), np.str_('Pine'): np.int64(10), np.str_('Red Oak'): np.int64(38), np.str_('Spruce'): np.int64(62)}

Applying SMOTE...
After SMOTE: (1114, 100) -> (2058, 100)
Scaling features...
Running grid search...
Grid completed in 6.0s
Best balanced accuracy (CV): 0.9441
Best parameters: {'C': 30, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}
Test accuracy: 0.7799
Test balanced accuracy: 0.6999

Testing: Multi_Aggregation
Features: ['enhanced_lbp']
Aggregation: ['mean', 'std']
Re-extracting training features...
Extracting features with types: ['enhanced_lbp']
Aggregation methods: ['mean', 'std']
Feature dimension per image: 54
Total feature d

Processing species: 100%|██████████| 7/7 [00:50<00:00,  7.25s/it]


Feature extraction complete!
Feature matrix shape: (1114, 108)
Species: [np.str_('Ash'), np.str_('Beech'), np.str_('Douglas Fir'), np.str_('Oak'), np.str_('Pine'), np.str_('Red Oak'), np.str_('Spruce')]
Label distribution: {np.str_('Ash'): np.int64(64), np.str_('Beech'): np.int64(264), np.str_('Douglas Fir'): np.int64(294), np.str_('Oak'): np.int64(36), np.str_('Pine'): np.int64(40), np.str_('Red Oak'): np.int64(162), np.str_('Spruce'): np.int64(254)}

Re-extracting test features...
Extracting features with types: ['enhanced_lbp']
Aggregation methods: ['mean', 'std']
Feature dimension per image: 54
Total feature dimension per tree: 108



Processing species: 100%|██████████| 7/7 [00:12<00:00,  1.74s/it]


Feature extraction complete!
Feature matrix shape: (268, 108)
Species: [np.str_('Ash'), np.str_('Beech'), np.str_('Douglas Fir'), np.str_('Oak'), np.str_('Pine'), np.str_('Red Oak'), np.str_('Spruce')]
Label distribution: {np.str_('Ash'): np.int64(14), np.str_('Beech'): np.int64(64), np.str_('Douglas Fir'): np.int64(72), np.str_('Oak'): np.int64(8), np.str_('Pine'): np.int64(10), np.str_('Red Oak'): np.int64(38), np.str_('Spruce'): np.int64(62)}

Applying SMOTE...
After SMOTE: (1114, 108) -> (2058, 108)
Scaling features...
Running grid search...
Grid completed in 6.6s
Best balanced accuracy (CV): 0.9164
Best parameters: {'C': 100, 'class_weight': None, 'gamma': 'scale', 'kernel': 'rbf'}
Test accuracy: 0.7127
Test balanced accuracy: 0.6221

FINAL RESULTS COMPARISON
    Configuration  Feature_Dim CV_Balanced_Acc Test_Accuracy Test_Balanced_Acc  Best_C Best_Gamma Class_Weight
     Enhanced_LBP           54          0.9339        0.7388            0.7244     100       auto         None
   